
# SDS 3386 2022 Fall Assignment 2: Data Wrangling

2022 Fall, Tanya Schmah

## Due Thursday October 6th at 6pm, on Brightspace

Total Marks: 10

This is a **group** assignment. You should complete it with two other people, preferably those you have chosen to work on a project with (though this is not required). By the time you submit your assignment, all group members should understand all answers and be able to explain them.

Start here: Download this file to your laptop, rename or make a copy of it with filename in the format: SDS3386_A2_Last1_Last2_Last3.ipynb
(In this class, last names are unique.)
Complete the assignment by editing the file. If group members work on different copies of the notebook, then when you're finished you must combine your notebooks into one "clean" copy for submission. When finished, export as HTML, and upload both the jupyter notebook and the HTML file to Brightspace. Each student should submit a copy of the same assignment to Brightspace, for easier marking.

You may copy and modify code from your Lab exercises, but if you copy more than two lines from another source, note the source in a comment.

# Question 1 (2 points)

Using the basketball data in "https://raw.githubusercontent.com/fivethirtyeight/data/master/nba-elo/nbaallelo.csv", determine the proportion of Home games ('game_location' = 'H') won, and the proportion of Away games won, over all the seasons of the NBA recorded in this dataset.

*Hint: run 'df.value_counts?'*

# Question 2 (4 points)

Use the dataset `grades.csv` provided with this assignment. 

Suppose the dataset contains grades out of 10 for each assignment and out of 100 for the exam. The grading policy for the course is:

    10% for each assignment and 70% for the final exam.

 

A) Read the data into a dataframe. Is this dataframe tidy? Justify your answer. If "No", then list all ways in which it is not tidy. 

B) Add a new column 'Final', calculated according to the above formula. Put 'INC' (for incomplete) for the student who didn't write the exam. Save the data to a file 'grades_calculated.csv', *without* an initial column containing row numbers (i.e. indexes).

*Hint: run 'df.to_csv?'*

C) Make a graphic showing all individual students' grades for all assessments. 



In [70]:
# A)
import pandas as pd
grades = pd.read_csv('grades.csv')
print(grades)

     Name  Asst1     Asst2  Asst3       Exam
0    Adam      9  8.000000    8.0  60.000000
1    Bing      7  8.000000    7.0  75.000000
2   Carla      6       NaN    5.0        NaN
3  Darius      6  7.000000   10.0  80.000000
4    mean      7  7.666667    7.5  71.666667


No the dataframe is not tidy. The assignment grades are not standardized to a specific precision. The title Asst can be modified to Assignment to clarify the headings. In addition, the mean is placed as a row instead of allowing to be computed as a result of the dataset, it must now be filtered out for accuracy.

In [71]:
# B)
import re
import numpy as np
# Create a grading policy that is adjustable
GRADING_POLICY = {
    'exam' : 0.7,
    'assignment': 0.1
}

assignment_column_regex = re.compile(r"Asst[0-9]+")


# Define a function 
def finalscore(row):
    final_grade = 0
    for column in grades:
        
        try:
            if assignment_column_regex.match(column) and pd.isna(row[column]):
                assignment_grade =  (row[column]/10)*(GRADING_POLICY['assignment'])
                final_grade += assignment_grade
            elif column == "Exam":
                if pd.isna(row[column]):
                    return "INC"
                exam_grade = (row[column]/100)*(GRADING_POLICY['exam'])
                final_grade = final_grade + exam_grade
        except Exception as e:
            return "INC"

    final_grade = np.round(final_grade*100, decimals=2)    
    return str(final_grade)

grades['Final'] = grades.apply(finalscore, axis=1)
grades = grades.round(decimals= 2)

print(grades)
print("Saving CSV")
grades.to_csv("grades_calculated.csv")
print("CSV Saved")


     Name  Asst1  Asst2  Asst3   Exam  Final
0    Adam      9   8.00    8.0  60.00   42.0
1    Bing      7   8.00    7.0  75.00   52.5
2   Carla      6    NaN    5.0    NaN    INC
3  Darius      6   7.00   10.0  80.00   56.0
4    mean      7   7.67    7.5  71.67  50.17
Saving CSV
CSV Saved


In [87]:
# C)
import altair as alt

columns_per_student = [column for column in grades if assignment_column_regex.match(column)]

columns_per_student.append('Exam')
columns_per_student.append('Final')


alt.Chart(grades, title="Student Grades").transform_fold(
    columns_per_student, 
    as_=['key_x', 'value_x']
).encode(
    x=alt.X('key_x:N', title=None),
    y=alt.Y('value_x:Q', title="Score (Weighted)"),
    facet=alt.Facet('Name')
).mark_bar().properties(width=200)


alt.Chart(...)

# Question 3 (4 points)

This question uses a larger subset of the Billboard Top 100 sets, in the dataset `billboardhw.csv` provided with this assignment. It is similar to the smaller subset that appeared in Question 6 in the Week 3 Lab.


A) Select all songs that first appeared on the charts in January 2000. (The date of first appearance is 'date_entered'.) 

*Hint: use pd.to_datetime()*

B) Partially tidy this chart, by putting all rankings (for all weeks) in one column named 'ranking'.

*Hint: use pd.melt() rather than wide_to_long().*

C) Produce a single graphic showing, for each song in the chart, a line graph of ranking versus time (week).